In [ ]:
import lmdb
from src.data import mdtraj_load, traj_to_confs
from mlcolvar.data.graph.utils import _create_dataset_from_configuration
from pathlib import Path

In [2]:
data_path = Path("/media/novelli/DATA/DESRES/DESRES-Trajectory_CLN025-0-protein/CLN025-0-protein")
trajectory_files = [str(traj) for traj in data_path.glob("*.dcd")]
top = next(data_path.glob("*.pdb")).__str__()
name = next(data_path.glob("*.pdb")).stem
traj = mdtraj_load(trajectory_files, top, 1)

In [5]:
system_selection = 'all and not type H'
configs, z_table, atom_names = traj_to_confs(
    traj, system_selection=system_selection
)

In [ ]:
cutoff = 7.0


In [9]:
data

Data(edge_index=[2, 11850], shifts=[11850, 3], unit_shifts=[11850, 3], positions=[166, 3], cell=[3, 3], node_attrs=[166, 4], n_system=[1, 1], n_env=[1, 1], weight=1.0)

In [12]:
database_path = Path().cwd() / "preprocessed_data"
if not database_path.exists():
    database_path.mkdir(parents=True)
lmdb_path = database_path / f"{name}-cutoff{int(cutoff)}.lmdb"
map_size = 10_995_116_277_760  # 1 TB
env = lmdb.open(lmdb_path.__str__(), map_size=map_size, subdir=False)

In [ ]:
import pickle
from tqdm import tqdm
def store_to_lmdb(env, configs, z_table, cutoff):
    """Store dictionary data to LMDB"""
    with env.begin(write=True) as txn:
        length = len(configs)
        # Store key metadata
        txn.put(b'__len__', pickle.dumps(length))
        for (idx, c) in tqdm(list(enumerate(configs))):
            value = _create_dataset_from_configuration(
                config=c, 
                z_table=z_table, 
                cutoff=cutoff, 
                buffer=0.0, 
            )
            item_key = f"item_{idx}".encode()
            txn.put(item_key, pickle.dumps(value))

In [15]:
store_to_lmdb(env, configs, z_table, cutoff)

  ━━━━━━━━━━━━━━━━━━━━━━━━━ 1/534743 Est. 33:42.0


KeyboardInterrupt: 